In [1]:
import sqlite3
import pandas as pd
from datetime import datetime
from datetime import date
import matplotlib.pyplot as plt

In [ ]:
# find your chat.db and establish a connection
conn = sqlite3.connect('/Users/<your user name here>/Library/Messages/chat.db')
cur = conn.cursor()

# query the database to get all the table names
cur.execute(" select name from sqlite_master where type = 'table' ")

for name in cur.fetchall():
    print(name)

In [3]:
## Getting required tables
messages = pd.read_sql_query('''select *, datetime(date/1000000000 + strftime("%s", "2001-01-01") ,"unixepoch","localtime")  as date_utc from message''', conn) 
handles = pd.read_sql_query("select * from handle", conn)
chat_message_joins = pd.read_sql_query("select * from chat_message_join", conn)

In [4]:
# these fields are only for ease of datetime analysis (e.g., number of messages per month or year)
messages['message_date'] = messages['date']
messages['timestamp'] = messages['date_utc'].apply(lambda x: pd.Timestamp(x))
messages['date'] = messages['timestamp'].apply(lambda x: x.date())
messages['month'] = messages['timestamp'].apply(lambda x: int(x.month))
messages['year'] = messages['timestamp'].apply(lambda x: int(x.year))

In [5]:
# rename the ROWID into message_id, because that's what it is
messages.rename(columns={'ROWID' : 'message_id'}, inplace = True)

In [6]:
# rename appropriately the handle and apple_id/phone_number as well
handles.rename(columns={'id' : 'contact_info', 'ROWID': 'handle_id'}, inplace = True)

In [ ]:
# merge the messages with the handles
merge_level_1 = pd.merge(messages[['text', 'handle_id', 'date','message_date' ,'timestamp', 'month','year','is_sent', 'message_id']],  handles[['handle_id', 'contact_info']], on ='handle_id', how='left')

# and then that table with the chats
df_messages = pd.merge(merge_level_1, chat_message_joins[['chat_id', 'message_id']], on = 'message_id', how='left')


# print(len(df_messages))
print(df_messages.head())

In [8]:
# for each unique chat_id, I measure the number of unique contacts.
# your contact does not count
chat_size = df_messages.groupby('chat_id')['contact_info'].nunique().to_frame().\
                    reset_index().rename(columns={'contact_info':'num_participants'})

# if there are at least 2 other contacts (other than you) then flag this as a group chat
chat_size['is_group_chat'] = chat_size['num_participants'].apply(lambda x: x>1)

df_messages = pd.merge(df_messages, chat_size[['chat_id','is_group_chat']], on='chat_id', how='left')

In [ ]:
# Adding contact names into data set
contact_grouping = {
    "<contact info name>":"<name you would like to add>",
    "+1234567890":"John Doe",
    # contact names wiped for privacy 

                   }

contact_df = pd.DataFrame(list(contact_grouping.items()), columns=['contact_info', 'contact_name'])

df_messages = pd.merge(df_messages,contact_df ,on='contact_info', how='left')
df_messages.head()

In [26]:
# for contacts with no name assigned to them, just have the contact as the name.
# this is helpful for grouping by name - 
# if you don't do this, and group by name, then all the NULL will be grouped into one
df_messages['contact_name'].fillna(df_messages['contact_info'],inplace=True)

In [19]:
# setting what characters I will allow in messagse
allowedChars = ' !$&,-.:;@()?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz1234567890' 
mySet = set(allowedChars)

In [33]:
# Open a text file in 'append' mode
with open('message_log.txt', 'a') as file:
    for _, row in df_messages.iterrows():
        
        if row.text and row.contact_name:    
            # Filter out characters not in mySet from row.text
            filtered_text = ''.join(char for char in row.text if char in mySet)

            # Filter out characters not in mySet from row.contact_name
            string_name = str(row.contact_name)
            filtered_contact_name = ''.join(char for char in string_name if char in mySet)

            # Create the message and customize for if sent or received
            message = ''
            if row.is_sent:
                message = f'To {filtered_contact_name}:\n{filtered_text}\n'
            else:
                message = f'From {filtered_contact_name}:\n{filtered_text}\n'
            
            # Add messages to file
            file.write(message)
            file.write('\n')